In [ ]:
 !pip install dstack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pydantic-duality to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 47.8 MB/s eta 0:00:00
  Created wheel for cursor: filename=cursor-1.3.5-py3-none-any.whl size=15823 sha256=793553ca615ee50638be6f3cc3b7d9d273c5512c3a798351915962eb1085dbd0
  Stored in director

In [4]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
import pandas as pd
import time
import os
import psutil
from transformers import AutoModelForCausalLM, AutoTokenizer

# === Settings ===
model_name = "microsoft/phi-1_5"
results_file = "runtime_results_phi_1_5.xlsx"

device_type = "cuda" if torch.cuda.is_available() else "cpu"
batch_sizes = [2, 4, 8, 16]

# === Load the model ===
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device_type)
model.eval()
config = model.config

# === Models features ===
num_layers = config.num_hidden_layers
hidden_size = config.hidden_size
vocab_size = config.vocab_size
activation_function = getattr(config, 'hidden_act', 'N/A')
max_position_embeddings = config.max_position_embeddings
num_heads = config.num_attention_heads
num_parameters = sum(p.numel() for p in model.parameters())
params_per_layer = num_parameters / num_layers
hidden_per_head = hidden_size / num_heads

# === Hardware features ===
num_cores = psutil.cpu_count(logical=True)

if device_type == "cuda":
    gpu_props = torch.cuda.get_device_properties(0)
    gpu_name = gpu_props.name
    gpu_memory = gpu_props.total_memory
else:
    gpu_name = "N/A"
    gpu_memory = 0

# === Prompts read ===
with open("prompts.txt", "r", encoding="utf-8") as f:
    prompts = [line.strip() for line in f if line.strip()]

# === Preprocessing ===
all_data = []

for prompt in prompts:
    encoded = tokenizer(prompt, return_tensors="pt")
    input_ids = encoded["input_ids"]
    sequence_length = input_ids.shape[1]

    for batch_size in batch_sizes:
        batch_input = input_ids.expand(batch_size, -1).to(device_type)

        if device_type == "cuda":
            torch.cuda.synchronize()
        start = time.time()
        with torch.no_grad():
            _ = model(batch_input)
        if device_type == "cuda":
            torch.cuda.synchronize()
        runtime_sec = time.time() - start

        data = {
            'model_name': model_name,
            'parameter_count': num_parameters,
            'num_layers': num_layers,
            'hidden_size': hidden_size,
            'sequence_length': sequence_length,
            'vocab_size': vocab_size,
            'max_position_embeddings': max_position_embeddings,
            'activation_function': activation_function,
            'model_type': 'gpt2',
            'params_per_layer': round(params_per_layer, 2),
            'hidden_per_head': round(hidden_per_head, 2),
            'prompt': prompt,
            'batch_size': batch_size,
            'device': device_type,
            'gpu_name': gpu_name,
            'gpu_memory_MB': gpu_memory // (1024 * 1024),
            'cpu_cores': num_cores,
            'runtime_sec': round(runtime_sec, 4),
        }

        all_data.append(data)
        print(f"Batch {batch_size} | Prompt '{prompt[:30]}...' done in {runtime_sec:.4f} sec")

# === Save Excel ===
df_new = pd.DataFrame(all_data)

if os.path.exists(results_file):
    df_old = pd.read_excel(results_file)
    df_all = pd.concat([df_old, df_new], ignore_index=True)
else:
    df_all = df_new

df_all.to_excel(results_file, index=False)
print("\n Saved to", results_file)


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Batch 2 | Prompt 'Hello, how are you today?...' done in 16.0444 sec
Batch 4 | Prompt 'Hello, how are you today?...' done in 11.4053 sec
Batch 8 | Prompt 'Hello, how are you today?...' done in 14.8937 sec
Batch 16 | Prompt 'Hello, how are you today?...' done in 30.1641 sec
Batch 2 | Prompt 'Explain quantum physics in sim...' done in 4.8255 sec
Batch 4 | Prompt 'Explain quantum physics in sim...' done in 8.1701 sec
Batch 8 | Prompt 'Explain quantum physics in sim...' done in 17.5757 sec
Batch 16 | Prompt 'Explain quantum physics in sim...' done in 34.4607 sec
Batch 2 | Prompt 'Translate the following senten...' done in 7.4200 sec
Batch 4 | Prompt 'Translate the following senten...' done in 14.0040 sec
Batch 8 | Prompt 'Translate the following senten...' done in 27.8246 sec
Batch 16 | Prompt 'Translate the following senten...' done in 57.4280 sec
Batch 2 | Prompt 'What are the benefits of medit...' done in 5.7765 sec
Batch 4 | Prompt 'What are the benefits of medit...' done in 10.8298 sec

In [ ]:
!dstack project add --name Fans-a-blazing --url https://sky.dstack.ai --token 15ec28bd-46bb-4cf0-8d45-c55a055b007d

In [ ]:
!git clone https://github.com/digispect-intel/lyceum_fans_a_blazing.git

Cloning into 'lyceum_fans_a_blazing'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 30 (delta 8), reused 26 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 226.26 KiB | 1.20 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
!cd lyceum_fans_a_blazing

In [ ]:
!dstack init

OK


In [ ]:
!ls ..

bin			    kaggle		      opt		 sys
boot			    lib			      proc		 tmp
content			    lib32		      python-apt	 tools
cuda-keyring_1.1-1_all.deb  lib64		      python-apt.tar.xz  usr
datalab			    libx32		      root		 var
dev			    media		      run
etc			    mnt			      sbin
home			    NGC-DL-CONTAINER-LICENSE  srv


In [ ]:
!cd lyceum_fans_a_blazing/ && git pull && dstack init && dstack apply -yf models/text_generation/gpt2/gpt2-cpu_2-mem_8.dstack.yml

Already up to date.
OK
⠧ Getting apply plan...
 Project          Fans-a-blazing                                          
 User             lizabespalova                                           
 Configuration    models/text_generation/gpt2/gpt2-cpu_2-mem_8.dstack.yml 
 Type             task                                                    
 Resources        cpu=2.. mem=8GB.. disk=50GB                             
 Spot policy      on-demand                                               
 Max price        -                                                       
 Retry policy     -                                                       
 Creation policy  reuse-or-create                                         
 Idle duration    5m                                                      
 Max duration     -                                                       
 Reservation      -                                                       

 #  BACKEND               RESOURCES              INS